## 高性能异步爬虫
目的：在爬虫中使用异步实现高性能的数据爬取操作


### 同步爬虫

In [2]:
import requests
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

# 这里的url，随便给几个就行，最好是数据长度较大的
# 效果明显，我这里是随便给的
url = {
    'https://img0.baidu.com/it/u=922902802,2128943538&fm=253&fmt=auto&app=120&f=JPEG?w=1422&h=800',
    'https://img0.baidu.com/it/u=925843206,3288141497&fm=253&fmt=auto&app=138&f=JPEG?w=500&h=769',
    'https://img1.baidu.com/it/u=3009731526,373851691&fm=253&fmt=auto&app=138&f=JPEG?w=800&h=500'
}

def get_content(url):
    print("正在爬取：",url)
    # get方法是一个阻塞的操作
    resp = requests.get(url=url, headers=headers)
    if resp.status_code == 200:
        return resp.content

def parse_content(content):
    print("响应数据长度为：",len(content))

for url in url:
    content = get_content(url)
    parse_content(content)

正在爬取： https://img0.baidu.com/it/u=925843206,3288141497&fm=253&fmt=auto&app=138&f=JPEG?w=500&h=769
响应数据长度为： 58640
正在爬取： https://img0.baidu.com/it/u=922902802,2128943538&fm=253&fmt=auto&app=120&f=JPEG?w=1422&h=800
响应数据长度为： 75683
正在爬取： https://img1.baidu.com/it/u=3009731526,373851691&fm=253&fmt=auto&app=138&f=JPEG?w=800&h=500
响应数据长度为： 36994


### 异步爬虫

#### 方式
- 多线程，多进程（不建议）  
  好处：可以为相关的操作的难度开启线程或者进程，阻塞操作就可以异步执行  
  弊端：无法无限制的开启多线程或者多进程
- 线程池、进程池（适当的）  
  好处：可以降低系统对进程或者线程创建和销毁的一个频率，从而很好的降低系统的开销  
  弊端：池中线程或进程的数量是有上限的
- 单线程+异步协程（推荐）  
  - event_loop：事件循环，相当于一个无限循环，我们可以把一些函数注册到这个事件循环上，当满足某些条件的时候，函数就会被循环执行。  
  - task：任务，它是对协程对象的进一步封装，包含了任务的各个状态。  
  - future：代表将来执行或还没有执行的任务，实际上和 task 没有本质区别。
  - async 定义一个协程
  - await 用来挂起阻塞方法的执行

In [3]:
# 爬取梨视频的例子
import requests
from lxml import etree
from multiprocessing.dummy import Pool
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

url = "https://www.pearvideo.com/category_1"


page_text = requests.get(url=url, headers=headers).text

tree = etree.HTML(page_text)

a_list = tree.xpath('//div[@id="listvideoList"]/ul[@id="listvideoListUl"]/li[@class="categoryem "]/div[@class="vervideo-bd"]/a/@href')
a_list = list(map(lambda x: "https://www.pearvideo.com/" + x, a_list))

name_list = tree.xpath('//div[@id="listvideoList"]/ul[@id="listvideoListUl"]/li[@class="categoryem "]/div[@class="vervideo-bd"]/a/div[2]/text()')

print(a_list)
print(name_list)

['https://www.pearvideo.com/video_1578274', 'https://www.pearvideo.com/video_1725093', 'https://www.pearvideo.com/video_1584102']
['她19岁被拐离家，因一封信再见家人', '湖北一小学校园内有座烈士墓，师生数十年默默守护祭扫', '老师借2万创业,他大三买车大四买房']


In [4]:
urls_end = []
for url_i in range(len(a_list)):
    # 拿到contID
    headers_mp4 = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
        # 防盗链：溯源，查看跳转前的网页
        'Referer': a_list[url_i]
    }
    contID = a_list[url_i].split("_")[1]
    videoStatusUrl = f"https://www.pearvideo.com/videoStatus.jsp?contId={contID}&mrd=0.46063027559272984"
    dic_mp4 = requests.get(url=videoStatusUrl, headers=headers_mp4).json()
    srcUrl = dic_mp4['videoInfo']['videos']['srcUrl']
    # 获取到的最后的视频的url
    systemTime = dic_mp4['systemTime']
    srcUrl = srcUrl.replace(systemTime, f"cont-{contID}")
    dic_res = {
        'name': '6-1' + name_list[url_i] + '.mp4',
        'url': srcUrl
    }
    print(dic_res)
    urls_end.append(dic_res)


{'name': '6-1她19岁被拐离家，因一封信再见家人.mp4', 'url': 'https://video.pearvideo.com/mp4/adshort/20190716/cont-1578274-11346226-182135_adpkg-ad_hd.mp4'}
{'name': '6-1湖北一小学校园内有座烈士墓，师生数十年默默守护祭扫.mp4', 'url': 'https://video.pearvideo.com/mp4/adshort/20210331/cont-1725093-15644312_adpkg-ad_hd.mp4'}
{'name': '6-1老师借2万创业,他大三买车大四买房.mp4', 'url': 'https://video.pearvideo.com/mp4/adshort/20190731/cont-1584102-13081995-113957_adpkg-ad_hd.mp4'}


##### 线程池

In [5]:
# 使用线程池对视频数据进行请求
def get_video_data(dic):
    url = dic['url']
    data = requests.get(url=url, headers=headers).content
    print(dic['name'],"开始下载！！")
    # 持久化存储操作
    with open(dic['name'], 'wb') as fp:
        fp.write(data)
        print(dic['name'],"下载成功！")
    pass

pool = Pool(3)
pool.map(get_video_data, urls_end)

pool.close()
pool.join()

6-1她19岁被拐离家，因一封信再见家人.mp4 开始下载！！
6-1她19岁被拐离家，因一封信再见家人.mp4 下载成功！
6-1湖北一小学校园内有座烈士墓，师生数十年默默守护祭扫.mp4 开始下载！！
6-1湖北一小学校园内有座烈士墓，师生数十年默默守护祭扫.mp4 下载成功！
6-1老师借2万创业,他大三买车大四买房.mp4 开始下载！！
6-1老师借2万创业,他大三买车大四买房.mp4 下载成功！


##### 异步协程
对于所有以async定义的函数，返回值是一个coroutine object  
详细可搜索python协程

In [ ]:

import asyncio

async def request(url):
    print("正在请求的url", url)
    print("请求成功",url)

# async修饰的函数，掉用之后返回的一个协程对象
c = request("www.baidu.com")

# 创建一个事件循环对象
loop = asyncio.get_event_loop()

# 将协程对象注册到loop中，然后启动loop
loop.run_until_complete(c)

